# ORB-facebook-comments-sentiment-analysis

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import nltk
import gradio as gr
import gradio
import joblib
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
#load csv file
Train = pd.read_csv("Posts_comment2.csv",names=['text','label'])
data=Train
data.head()

In [ ]:
aforstopwords = pd.read_csv("aforstopwords.txt")
aforstopwords.head()

In [ ]:
test_data = pd.read_csv("Posts_comment_test.csv",names=['text'])
test_data.head()

In [ ]:
#Removing Punctuations, Numbers, and Special Characters
#[a-zA-Z] = Any single character in the range a-z or A-Z
# ^ = Start of line 
# $ = End of line 

data['text'] = data['text'].str.replace("[^a-zA-Z#]", " ")
data['text'] = data['text'].str.replace("#", "")
#data.head()

In [ ]:
test_data['text'] = test_data['text'].str.replace("[^a-zA-Z#]", " ")
test_data['text'] = test_data['text'].str.replace("#", "")
#test_data.head()

In [ ]:
# get most common words in training dataset
from collections import Counter 
all_words = []
for line in list(data['text']):
    words = line.split()
    for word in words:
        all_words.append(word.lower())
    
    
a=Counter(all_words).most_common(10)
a

In [ ]:
#tokenization
data['text'] = data['text'].apply(lambda x: x.split())
data.head()

In [ ]:
test_data['text'] = test_data['text'].apply(lambda x: x.split())
test_data.head()

In [ ]:
#stemmer
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

data['text']= data['text'].apply(lambda x: [stemmer.stem(i) for i in x])
data.head(10)

In [ ]:
test_data['text']= test_data['text'].apply(lambda x: [stemmer.stem(i) for i in x])
test_data.head()

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
import string 

def process(text):
    # Check characters to see if they are in punctuation
    nopunc = set(char for char in list(text) if char not in string.punctuation)
    # Join the characters to form the string.
    nopunc = " ".join(nopunc)
    # remove any stopwords if present
    return [word for word in nopunc.lower().split() if word.lower() not in aforstopwords]
    

In [ ]:
data['text'] = data['text'].apply(process) 
data.head()

In [ ]:
# test_data
test_data['text'] = test_data['text'].apply(process) 
test_data.head()

In [ ]:
def string (text):
    to_return=""
    for i in list(text):
        to_return += str(i) + " "
    to_return = to_return[:-1]
    
    return to_return
    
       
data['text'] = data['text'].apply(string)
data.head()  

In [ ]:
test_data['text'] = test_data['text'].apply(string)
test_data.head()

In [ ]:
#Split data into training and testing sets 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data["text"], 
                                                    data["label"], test_size = 0.2, random_state = 42)

print("training set :",x_train.shape,y_train.shape)
print("testing set :",x_test.shape,y_test.shape)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)


In [ ]:
test_x = test_data['text']
test_x

In [ ]:
x_train_counts = count_vect.fit_transform(x_train)
x_train_tfidf = transformer.fit_transform(x_train_counts)

print(x_train_counts.shape)
print(x_train_tfidf.shape)

In [ ]:
x_train.head()

In [ ]:
test_x_counts = count_vect.transform(test_x)
test_x_tfidf = transformer.transform(test_x_counts)

print(test_x_counts.shape)
print(test_x_tfidf.shape)

In [ ]:
test_x_counts

In [ ]:
x_test_counts = count_vect.transform(x_test)
x_test_tfidf = transformer.transform(x_test_counts)

print(x_test_counts.shape)
print(x_test_tfidf.shape)

#Model building

#Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=200)
model.fit(x_train_tfidf,y_train)

In [ ]:
predictions = model.predict(x_test_tfidf)

In [ ]:
submission  = model.predict(test_x_tfidf)

In [ ]:
#Accuracy_score
from sklearn.metrics import accuracy_score
rfac=accuracy_score(y_test,predictions)*100

In [ ]:
rfac

In [ ]:
tokenizer = Tokenizer(num_words=5000)

In [ ]:
def rfpredict_sentiment(text):
    fbcom = tokenizer.texts_to_sequences([text])
    fbcom = pad_sequences(fbcom,maxlen=1355)
    prediction = int(model.predict(fbcom).round().item())
    #return prediction
    if prediction==1:
        return "Predicted label: Posetive"
    else:
        return "Predicted label: Negative"
    #print("Predicted label: ",[prediction])

In [ ]:
outputs = gr.outputs.Textbox()
app = gr.Interface(fn=rfpredict_sentiment, 
                   inputs='text', 
                   outputs=gradio.outputs.Textbox(),
                   description="This is sentment anlysis of ORB customer comments")

In [ ]:
app.launch()